In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv()) # important line if cannot load api key

# Getting the api keys from the .env file

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["SERPER_API_KEY"] = os.getenv('SERPER_API_KEY')

# load tools
serper_search_tool = SerperDevTool()
scrap_tool = ScrapeWebsiteTool()
website_search_tool = WebsiteSearchTool()


In [ ]:
# local LLM
from langchain_community.llms import Ollama

llm = Ollama(model="llama3")

In [ ]:
from crewai import Agent, Task, Crew
from crewai.process import Process

tools=[serper_search_tool, scrap_tool, website_search_tool]

# Define the Agents
travel_expert = Agent(
    role='Hotel Researcher',
    goal='Efficiently locate and compile a comprehensive list of suitable hotel options in the specified location, \
        adhering to the given criteria such as check-in/out dates, number of guests, and any other specific requirements. \
        Ensure the gathered information is accurate, up-to-date, and includes key details like pricing, amenities, and guest ratings.',
    backstory='You are an experienced digital travel concierge with a keen eye for detail and a passion for finding the perfect accommodations.\
        With years of experience in the hospitality industry and a vast knowledge of global hotel chains and boutique properties, you\
        have honed your skills in navigating various booking platforms and hotel databases. \
        Your expertise lies in quickly sifting through numerous options to identify the most suitable choices for travelers, \
        taking into account factors such as location, price, amenities, and guest reviews.',
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=tools,
)

hotel_reviewer = Agent(
    role='Hotel Quality Analyst',
    goal='Thoroughly evaluate and compare hotel options to provide detailed, unbiased assessments that help travelers make informed decisions. \
        Analyze each hotel\'s amenities, location, value for money, and guest experiences to create comprehensive reviews that highlight strengths, weaknesses, and unique features.',
    backstory='You are a seasoned travel industry professional with over a decade of experience in hotel evaluation and critique. \
        Your background includes working as a luxury hotel inspector, a travel journalist for renowned publications, and a consultant for hotel rating systems. \
        This diverse experience has honed your ability to assess accommodations from multiple perspectives, considering both objective criteria and subjective guest experiences. \
        Your reviews are known for their depth, fairness, and ability to capture the essence of each property. You have a particular talent for identifying hidden gems and spotting potential issues that might affect a guest\'s stay. \
        Your expertise covers a wide range of accommodations, from budget-friendly options to ultra-luxury resorts, and you\'re adept at evaluating hotels in various cultural contexts around the world.',
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=tools,
)

In [ ]:
# Define the Tasks
hotel_search_task = Task(
    description="Search for 5 hotels in {location} for {number_of_people} adult people, checking in on {check_in} and checking out on {check_out}.",
    agent=travel_expert,
    expected_output="All the details of a specifically chosen accommodation.")

hotel_review_task = Task(
    description="Based on the recommendations provided, pick the best options based on ratings, reviews, and facilities available. \
    Consider that Budget is {budget} CAD. Try to find accommodations in and around the Budget.",
    expected_output="All the details of a specifically chosen accommodation including the price, URL, and any image if available.",
    agent=hotel_reviewer,
)

In [ ]:
# Create the Crew
travel_agent_crew = Crew(
    agents=[travel_expert, hotel_reviewer],
    tasks=[hotel_search_task, hotel_review_task],
    verbose=True,
    process=Process.sequential,
    # step_callback=[conversation_logger]
)

event_criteria = {
    'location': 'Brampton, Ontario',
    'check_in': '23rd August, 2024',
    'check_out': '25th August, 2024',
    'number_of_people': 2,
    'budget': 600
    }

result = travel_agent_crew.kickoff(inputs=event_criteria)

2024-07-12 14:08:21,591 - 8379960320 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-07-12 14:08:21][DEBUG]: == Working Agent: Hotel Researcher
 [2024-07-12 14:08:21][INFO]: == Starting Task: Search for 5 hotels in Brampton, Ontario for 2 adult people, checking in on 23rd August, 2024 and checking out on 25th August, 2024.


> Entering new CrewAgentExecutor chain...
A new challenge! Let's get started!

Thought: I need to gather information about hotels in Brampton, Ontario for 2 adult people, with a check-in date of August 23rd, 2024 and a check-out date of August 25th, 2024.

Action: Search the internet

Action Input: {search_query: "hotels in Brampton, Ontario", 


Search results: Title: Top Hotels in Brampton from $112
Link: https://www.hotels.com/de167871/hotels-brampton-ontario/
Snippet: Where to stay in Brampton? · Monte Carlo Inn - Brampton Suites · Hyatt Place Toronto - Brampton · Fairfield Inn and Suites by Marriott Toronto Brampton.
---
Title: Top Hotels in Brampton from $110 - Fully Refundable Options
Link: https://www.expedia.com/Brampton-Hotels.d680

In [ ]:
print(result)

Based on the search results from various websites, the top hotels in Brampton that fit within the budget of $600 CAD are:

1. Hyatt Place Toronto-Brampton
2. Hilton Garden Inn Toronto/Brampton
3. Fairmont Royal York (with refundable options)
4. Cambridge Suites Toronto
5. Holiday Inn Toronto/Brampton

These hotels offer a range of amenities, including tea/coffee making equipment, flat-screen TVs with satellite channels, and modern conveniences. They also have high ratings from travelers and are located in the heart of Brampton.
